In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## AI 허브 한국어 대화(의류) 데이터 활용 예제: Seq2Seq 구현 및 학습
- AI 허브 한국어 대화(의류) 데이터셋을 이용한 문장 생성

In [2]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 11.0 MB/s 
     |████████████████████████████████| 448 kB 21.4 MB/s 


In [3]:
import random
import tensorflow as tf
from konlpy.tag import Okt

## 하이퍼 파라미터

In [4]:
EPOCHS = 200
NUM_WORDS = 10000 #Size of the vocabulary

## Encoder

In [5]:
class Encoder(tf.keras.Model):
    def __init__(self):
        super(Encoder, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64) #원-핫 인코딩 형태로 입력됨
        self.lstm = tf.keras.layers.LSTM(512, return_state=True) #return_state=True 해주어어야 h(hidden state), c(cell state) 얻을 수 있음
        # GRU: cell state가 없으므로 hidden state값만 추출

    def call(self, x, training=False, mask=None):
        x = self.emb(x)
        _, h, c = self.lstm(x)
        return h, c

## Decoder

In [6]:
class Decoder(tf.keras.Model):
    def __init__(self):
        super(Decoder, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True) # return_sequences=True: 출력된 모든 단어를 알아야하기 때문에
        self.dense = tf.keras.layers.Dense(NUM_WORDS, activation='softmax')

    def call(self, inputs, training=False, mask=None):
        x, h, c = inputs
        x = self.emb(x)
        x, h, c = self.lstm(x, initial_state=[h, c])
        return self.dense(x), h, c

## Seq2seq

In [7]:
class Seq2seq(tf.keras.Model): # Encoder + Decoder
    def __init__(self, sos, eos):
        super(Seq2seq, self).__init__()
        self.enc = Encoder()
        self.dec = Decoder()
        self.sos = sos # start of sequence
        self.eos = eos # end of sequence

    def call(self, inputs, training=False, mask=None):
        # 학습
        if training is True:
            x, y = inputs
            h, c = self.enc(x)
            y, _, _ = self.dec((y, h, c))
            return y

        # 테스트
        else: 
            x = inputs
            h, c = self.enc(x)
            y = tf.convert_to_tensor(self.sos) # 첫번째 입력으로 `sos`
            y = tf.reshape(y, (1, 1))

            seq = tf.TensorArray(tf.int32, 64) # 64길이

            # 마지막으로 얻은 출력을 다시 입력으로 넣어주기 위해 for문을 사용
            for idx in tf.range(64): 
                y, h, c = self.dec([y, h, c]) # 이때 y값은 softmax를 거친 값으로 one-hot 벡터로 바꾸고 spare 표현으로 바꾸기 위해 argmax 취해줌 
                y = tf.cast(tf.argmax(y, axis=-1), dtype=tf.int32)
                y = tf.reshape(y, (1, 1)) # 배치를 표현하기 위해 (1,1)으로 reshape
                seq = seq.write(idx, y)

                if y == self.eos:
                    break

            return tf.reshape(seq.stack(), (1, 64))

## 학습, 테스트 루프 정의

In [8]:
# Implement training loop
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    output_labels = labels[:, 1:]
    shifted_labels = labels[:, :-1] # 학습할 때 Decoder의 입력으로 들어갈 y
    with tf.GradientTape() as tape:
        predictions = model([inputs, shifted_labels], training=True)
        loss = loss_object(output_labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(output_labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, inputs):
    return model(inputs, training=False)

## 데이터셋 준비

In [9]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/NLP practice/data/aihub_한국어대화_의류.csv')
data.head()

,SPEAKER,SENTENCE,DOMAINID,DOMAIN,CATEGORY,SPEAKERID,SENTENCEID,MAIN,SUB,QA,QACNCT,MQ,SQ,UA,SA,개체명,용어사전,지식베이스,Unnamed: 18
0,고객,신발은 여기 있는 게 다예요?,B,의복의류점,신발,1,1,종류별신발제품문의요청,NaN,Q,NaN,신발은 여기 있는 게 다예요?,NaN,NaN,NaN,"신발, 여기",NaN,신발/제품,0.0
1,점원,네 성인이나 아동 다 있어요,B,의복의류점,신발,0,2,종류별신발제품문의요청,NaN,A,NaN,NaN,NaN,NaN,네 성인이나 아동 다 있어요,"성인, 아동",NaN,"성인/대상, 아동/대상",NaN
2,점원,발 사이즈 몇 신으세요?,B,의복의류점,신발,0,3,종류별신발제품문의요청,사이즈,Q,NaN,NaN,발 사이즈 몇 신으세요?,NaN,NaN,"발, 사이즈",NaN,NaN,NaN
3,고객,230이요,B,의복의류점,신발,1,4,종류별신발제품문의요청,사이즈,A,NaN,NaN,NaN,230이요,NaN,230,NaN,230/사이즈,NaN
4,점원,편하게 신을 수 있는 거 찾으세요?,B,의복의류점,신발,0,5,착화감,제품문의,Q,NaN,NaN,편하게 신을 수 있는 거 찾으세요?,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Q-A로 이루어진 데이셋으로 정제
QA_idx = []
for idx in range(len(data)):
    if idx != len(data)-1:
        if data.loc[idx]['QA'] == 'Q' and data.loc[idx+1]['QA'] == 'A':
            QA_idx.append(idx)
            QA_idx.append(idx+1)
            idx = idx + 1


In [11]:
data = data.loc[QA_idx]
data.tail()

,SPEAKER,SENTENCE,DOMAINID,DOMAIN,CATEGORY,SPEAKERID,SENTENCEID,MAIN,SUB,QA,QACNCT,MQ,SQ,UA,SA,개체명,용어사전,지식베이스,Unnamed: 18
14895,점원,그 옷은 십이만 팔천 원이에요,B,의복의류점,의류,0,22,가격문의,NaN,A,NaN,NaN,NaN,NaN,그 옷은 십이만 팔천 원이에요,"옷,십이만 팔천 원",NaN,십이만 팔천 원/가격,NaN
14896,고객,지금 봄옷은 이르지 않나요?,B,의복의류점,의류,1,23,계절상품문의,NaN,Q,NaN,지금 봄옷은 이르지 않나요?,NaN,NaN,NaN,봄옷,NaN,봄옷/의류,NaN
14897,점원,지금 봄옷 입기 괜찮아요,B,의복의류점,의류,0,24,계절상품문의,NaN,A,NaN,NaN,NaN,NaN,지금 봄옷 입기 괜찮아요,봄옷,NaN,봄옷/의류,NaN
14898,고객,요즘 유행하는 색깔이 뭐예요?,B,의복의류점,의류,1,25,인기제품문의,NaN,Q,NaN,요즘 유행하는 색깔이 뭐예요?,NaN,NaN,NaN,"유행,색깔",NaN,NaN,NaN
14899,점원,요즘 파스텔 톤이 유행이에요,B,의복의류점,의류,0,26,인기제품문의,NaN,A,NaN,NaN,NaN,NaN,요즘 파스텔 톤이 유행이에요,"파스텔 톤,유행",NaN,파스텔 톤/색,NaN


In [12]:
okt = Okt()

QA_data = data['SENTENCE']
seq = [' '.join(okt.morphs(line))+' \n' for line in QA_data]

questions = seq[::2] 
answers = ['\t ' + lines for lines in seq[1::2]] # '\t': sos로 사용

num_sample = len(questions)

perm = list(range(num_sample)) # 데이터가 편향된 상태기때문에 섞음
random.seed(0)
random.shuffle(perm)

train_q = list()
train_a = list()
test_q = list()
test_a = list()

for idx, qna in enumerate(zip(questions, answers)):
    q, a = qna
    if perm[idx] > num_sample//5: # 데이터의 4/5
        train_q.append(q)
        train_a.append(a)
    else: # 데이터의 1/5
        test_q.append(q)
        test_a.append(a)

# tokenizer: 각 단어를 숫자로 변형
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS, # filters에 해당하는 문자는 제거 ('\n'과 '\t'는 제외)
                                                  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~')

tokenizer.fit_on_texts(train_q + train_a)

# 숫자로 된 단어의 나열
train_q_seq = tokenizer.texts_to_sequences(train_q)
train_a_seq = tokenizer.texts_to_sequences(train_a)

test_q_seq = tokenizer.texts_to_sequences(test_q)
test_a_seq = tokenizer.texts_to_sequences(test_a)

x_train = tf.keras.preprocessing.sequence.pad_sequences(train_q_seq,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=64)
y_train = tf.keras.preprocessing.sequence.pad_sequences(train_a_seq,
                                                        value=0,
                                                        padding='post', #출력 데이터를 앞쪽으로 두기위해 뒤에 패딩
                                                        maxlen=65) #'\t'와 '\n'가 붙어있는 상황이라 앞에 하나 떼고 사용하고, 뒤에 하나 떼고 사용 => 실제 길이는 64로 사용


x_test = tf.keras.preprocessing.sequence.pad_sequences(test_q_seq,
                                                       value=0,
                                                       padding='pre',
                                                       maxlen=64)
y_test = tf.keras.preprocessing.sequence.pad_sequences(test_a_seq,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=65)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1).prefetch(1024)

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [13]:
# Create model
model = Seq2seq(sos=tokenizer.word_index['\t'],
                eos=tokenizer.word_index['\n'])

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## 학습 루프 동작

In [14]:
from tqdm.notebook import tqdm

for epoch in tqdm(range(EPOCHS)):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100))

    train_loss.reset_states()
    train_accuracy.reset_states()

  0%|          | 0/200 [00:00<?, ?it/s]

Epoch 1, Loss: 1.0648168325424194, Accuracy: 89.67798614501953
Epoch 2, Loss: 0.5948834419250488, Accuracy: 91.4739990234375
Epoch 3, Loss: 0.5452613830566406, Accuracy: 91.96234893798828
Epoch 4, Loss: 0.5253956317901611, Accuracy: 92.08804321289062
Epoch 5, Loss: 0.5078579783439636, Accuracy: 92.2298812866211
Epoch 6, Loss: 0.49200260639190674, Accuracy: 92.38900756835938
Epoch 7, Loss: 0.4774836301803589, Accuracy: 92.57408905029297
Epoch 8, Loss: 0.4604460299015045, Accuracy: 92.73754119873047
Epoch 9, Loss: 0.4439573884010315, Accuracy: 92.90331268310547
Epoch 10, Loss: 0.4258338510990143, Accuracy: 93.0243911743164
Epoch 11, Loss: 0.4089261293411255, Accuracy: 93.18266296386719
Epoch 12, Loss: 0.39179450273513794, Accuracy: 93.29825592041016
Epoch 13, Loss: 0.37579813599586487, Accuracy: 93.40982055664062
Epoch 14, Loss: 0.3595995008945465, Accuracy: 93.515625
Epoch 15, Loss: 0.3432786762714386, Accuracy: 93.64420318603516
Epoch 16, Loss: 0.32711470127105713, Accuracy: 93.7860336

## 테스트 루프

In [15]:
for idx, (test_seq, test_labels) in enumerate(test_ds):
    if idx > 50:
        break
    prediction = test_step(model, test_seq)
    test_text = tokenizer.sequences_to_texts(test_seq.numpy())
    gt_text = tokenizer.sequences_to_texts(test_labels.numpy())
    texts = tokenizer.sequences_to_texts(prediction.numpy())
    print('_')
    print('q: ', test_text)
    print('a: ', gt_text)
    print('p: ', texts)


# 문장에 단어들이 없거나 끊어진 이유는 tokenizer에서 10000개의 word만 사용했기 때문에 이 word에 해당되지 않는 경우 제거되기 때문!

_
q:  ['편하게 신 을 수 있는 거 찾으세요 \n']
a:  ['\t 네 봄 이니까 편하게 신 을 수 있는 거 \n']
p:  ['혼자 천천히 둘러볼게요 \n']
_
q:  ['굽 좀 높은 거 없나요 \n']
a:  ['\t 봄 상품 은 아직 어른 제품 이 많이 안 나왔습니다 \n']
p:  ['3 번 피팅룸 이 비어있으니 들어가세요 \n']
_
q:  ['또 안 들어와요 \n']
a:  ['\t 네 이건 다 끝났어요 \n']
p:  ['다음 주 에 들어와요 \n']
_
q:  ['이 거 는 천이 죠 \n']
a:  ['\t 네 맞아요 \n']
p:  ['네 공단 천 원 이에요 \n']
_
q:  ['며칠 까지 휴무 예요 \n']
a:  ['\t 설 까지 쉬 고 다음 날 열 거 같아요 \n']
p:  ['9시 까지 해 요 \n']
_
q:  ['이 거 예요 다 돌 인가요 \n']
a:  ['\t 도 있고 도자기 도 있어요 \n']
p:  ['네 이 쪽 은 입학생 들용 이건 초등 아니고 좀 뜨기 때문 에 불편한 점 이 있습니다 \n']
_
q:  ['몇 시 에 문 닫아요 \n']
a:  ['\t 8시 까지 합니다 \n']
p:  ['9시 50분 이요 \n']
_
q:  ['원단 은 뭐 예요 \n']
a:  ['\t 원단 이 구김 안 가고 참 괜찮아요 \n']
p:  ['그거 입생로랑 휴대폰 케이스 인데 가죽 이에요 \n']
_
q:  ['브로치 같은 건 어디 있나요 \n']
a:  ['\t 여기 있는 거 밖에 없어요 \n']
p:  ['그레이 는 지금 재고 가 다 나갔어요 \n']
_
q:  ['온누리 상품권 도 되죠 \n']
a:  ['\t 네 됩니다 \n']
p:  ['네 생활 방수 다 돼요 \n']
_
q:  ['안 에 하면 안 떨어져요 \n']
a:  ['\t 안 떨어져요 \n']
p:  ['네 이건 국산 이에요 \n']
_
q:  ['얼마 예요 \n']
a:  ['\t 핀 거 는 좀 싼 거고 이 거 는 3만 원대 예요 \n']
p:  ['정가 는 

## AI 허브 한국어 대화(의류) 데이터 활용 예제: Attention 매커니즘 구현 및 학습
- Seq2Seq 모델의 정확도 및 성능과 Attention 매커니즘을 활용한 모델과의 성능 차이를 비교해보자

## Encoder with Attention Mechanism

In [16]:
class Encoder(tf.keras.Model):
    def __init__(self):
        super(Encoder, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64) 
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True) 
        # Attention 매커니즘 적용할 때 key, value는 encoder에서 나오는 hidden state를 전부 활용했었어야 했음
        # 그러기 위해 모든 hidden state를 sequences 형태로 출력해주기 위해 returnn_sequences=True 가 추가됨

    def call(self, x, training=False, mask=None):
        x = self.emb(x)
        H, h, c = self.lstm(x)
        return H, h, c

## Decoder with Attention Mechanism

In [17]:
class Decoder(tf.keras.Model):
    def __init__(self):
        super(Decoder, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 64)
        # attention mechanism: LSTM 출력에 Attention value를 concatenate해서 dense layer로 넘겨주는 것
        self.lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True)
        self.att = tf.keras.layers.Attention() # attention mechanism
        self.dense = tf.keras.layers.Dense(NUM_WORDS, activation='softmax')

    def call(self, inputs, training=False, mask=None):
        y_, s0, c0, H = inputs # y_: shifted output, s0과 c0: 처음 들어오는 decoder단의 hidden state와 cell state
        y_ = self.emb(y_)
        # S: 모든 hidden state 받아온 것
        S, h, c = self.lstm(y_, initial_state=[s0, c0]) # shifted output과 initial_state를 입력받아 모든 sequence의 출력을 내줌
        
        # S를 query로 사용, Encoder단에서 받아온 H는 key와 value로 사용
        # query로 사용할 때는 S를 그대로 사용하는 것이 아니라, 한 time-step 앞선 것을 사용해야 됨!!
        S_ = tf.concat([s0[:, tf.newaxis, :], S[:, :-1, :]], axis=1) 
        # s0[:, tf.newaxis, :] : 맨 처음에 들어오는 건 길이가 1이라 2차원 형태 -> 이를 3차원으로 확장
        # S[:, :-1, :]] : 마지막 hidden state껀 배제하여 총 길이가 64라면 63개만 사용 => 1+63 = 총 64의 길이
        A = self.att([S_, H])
        # attention value를 dense layer로 넣어주기 전에 S(hidden state)에 A(attention value)를 concatenate
        y = tf.concat([S, A], axis=-1)

        return self.dense(y), h, c

## Seq2Seq with Attention Mechanism

In [18]:
class Seq2seq(tf.keras.Model): # Encoder + Decoder
    def __init__(self, sos, eos):
        super(Seq2seq, self).__init__()
        self.enc = Encoder()
        self.dec = Decoder()
        self.sos = sos # start of sequence
        self.eos = eos # end of sequence

    def call(self, inputs, training=False, mask=None):
        # 학습
        if training is True:
            x, y = inputs
            H, h, c = self.enc(x)
            y, _, _ = self.dec((y, h, c, H))
            return y

        # 테스트
        else: 
            x = inputs
            H, h, c = self.enc(x)
            y = tf.convert_to_tensor(self.sos) # 첫번째 입력으로 `sos`
            y = tf.reshape(y, (1, 1))

            seq = tf.TensorArray(tf.int32, 64) # 64길이

            # 마지막으로 얻은 출력을 다시 입력으로 넣어주기 위해 for문을 사용
            for idx in tf.range(64): 
                y, h, c = self.dec([y, h, c, H]) # 이때 y값은 softmax를 거친 값으로 one-hot 벡터로 바꾸고 spare 표현으로 바꾸기 위해 argmax 취해줌 
                y = tf.cast(tf.argmax(y, axis=-1), dtype=tf.int32)
                y = tf.reshape(y, (1, 1)) # 배치를 표현하기 위해 (1,1)으로 reshape
                seq = seq.write(idx, y)

                if y == self.eos:
                    break

            return tf.reshape(seq.stack(), (1, 64))

## 학습, 테스트 루프 정의

In [19]:
# Implement training loop
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    output_labels = labels[:, 1:]
    shifted_labels = labels[:, :-1] # 학습할 때 Decoder의 입력으로 들어갈 y
    with tf.GradientTape() as tape:
        predictions = model([inputs, shifted_labels], training=True)
        loss = loss_object(output_labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(output_labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, inputs):
    return model(inputs, training=False)

In [20]:
okt = Okt()

QA_data = data['SENTENCE']
seq = [' '.join(okt.morphs(line))+' \n' for line in QA_data]

questions = seq[::2] 
answers = ['\t ' + lines for lines in seq[1::2]] # '\t': sos로 사용

num_sample = len(questions)

perm = list(range(num_sample)) # 데이터가 편향된 상태기때문에 섞음
random.seed(0)
random.shuffle(perm)

train_q = list()
train_a = list()
test_q = list()
test_a = list()

for idx, qna in enumerate(zip(questions, answers)):
    q, a = qna
    if perm[idx] > num_sample//5: # 데이터의 4/5
        train_q.append(q)
        train_a.append(a)
    else: # 데이터의 1/5
        test_q.append(q)
        test_a.append(a)

# tokenizer: 각 단어를 숫자로 변형
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS, # filters에 해당하는 문자는 제거 ('\n'과 '\t'는 제외)
                                                  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~')

tokenizer.fit_on_texts(train_q + train_a)

# 숫자로 된 단어의 나열
train_q_seq = tokenizer.texts_to_sequences(train_q)
train_a_seq = tokenizer.texts_to_sequences(train_a)

test_q_seq = tokenizer.texts_to_sequences(test_q)
test_a_seq = tokenizer.texts_to_sequences(test_a)

x_train = tf.keras.preprocessing.sequence.pad_sequences(train_q_seq,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=64)
y_train = tf.keras.preprocessing.sequence.pad_sequences(train_a_seq,
                                                        value=0,
                                                        padding='post', #출력 데이터를 앞쪽으로 두기위해 뒤에 패딩
                                                        maxlen=65) #'\t'와 '\n'가 붙어있는 상황이라 앞에 하나 떼고 사용하고, 뒤에 하나 떼고 사용 => 실제 길이는 64로 사용


x_test = tf.keras.preprocessing.sequence.pad_sequences(test_q_seq,
                                                       value=0,
                                                       padding='pre',
                                                       maxlen=64)
y_test = tf.keras.preprocessing.sequence.pad_sequences(test_a_seq,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=65)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32).prefetch(1024)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1).prefetch(1024)

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [21]:
# Create model
model = Seq2seq(sos=tokenizer.word_index['\t'],
                eos=tokenizer.word_index['\n'])

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

## 학습 루프


In [22]:
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100))

    train_loss.reset_states()
    train_accuracy.reset_states()

Epoch 1, Loss: 0.9920271635055542, Accuracy: 89.816650390625
Epoch 2, Loss: 0.5803816318511963, Accuracy: 91.8026351928711
Epoch 3, Loss: 0.542046070098877, Accuracy: 92.11485290527344
Epoch 4, Loss: 0.5167677402496338, Accuracy: 92.28753662109375
Epoch 5, Loss: 0.49235090613365173, Accuracy: 92.48097229003906
Epoch 6, Loss: 0.46798476576805115, Accuracy: 92.65538787841797
Epoch 7, Loss: 0.445870965719223, Accuracy: 92.81855010986328
Epoch 8, Loss: 0.4247305393218994, Accuracy: 92.97970581054688
Epoch 9, Loss: 0.40266188979148865, Accuracy: 93.14315795898438
Epoch 10, Loss: 0.38166549801826477, Accuracy: 93.28845977783203
Epoch 11, Loss: 0.360933780670166, Accuracy: 93.43260192871094
Epoch 12, Loss: 0.3378351330757141, Accuracy: 93.58366394042969
Epoch 13, Loss: 0.31380921602249146, Accuracy: 93.83706665039062
Epoch 14, Loss: 0.28822365403175354, Accuracy: 94.16397857666016
Epoch 15, Loss: 0.2640449106693268, Accuracy: 94.5358657836914
Epoch 16, Loss: 0.23959213495254517, Accuracy: 94.

## 테스트 루프

In [25]:
len(test_ds)

1356

In [30]:
for idx, (test_seq, test_labels) in enumerate(test_ds):
    if idx > 200 and idx < 250:
        prediction = test_step(model, test_seq)
        test_text = tokenizer.sequences_to_texts(test_seq.numpy())
        gt_text = tokenizer.sequences_to_texts(test_labels.numpy())
        texts = tokenizer.sequences_to_texts(prediction.numpy())
        print('_')
        print('q: ', test_text)
        print('a: ', gt_text)
        print('p: ', texts)


_
q:  ['이 거 는 얼 만 데 요 \n']
a:  ['\t 만 천 원 입니다 \n']
p:  ['이 거 처음 에 나왔을 때 십이만 원 받은 건데 팔만 원 이요 \n']
_
q:  ['엄마 들 입 을 건데 요 좀 밝은 색 으로 요 \n']
a:  ['\t 겨울 옷 이라서 이 정도 가 밝은 편이 에요 \n']
p:  ['저 게 면 이 게 잘 나가요 \n']
_
q:  ['바꾸러 오면 좀 그렇겠죠 \n']
a:  ['\t 교환 은 됩니다 \n']
p:  ['네 들 한테 나와서 안 에 얇게 고 추천 해 요 \n']
_
q:  ['이 거 드라이 해야 해 요 \n']
a:  ['\t 드라이 하면 좋지만 물 세탁 도 돼요 \n']
p:  ['울 샴푸 로 물 에 몇 방울 떨어뜨려서 조물조물 하면 됩니다 \n']
_
q:  ['색깔 은 뭐 있어요 \n']
a:  ['\t 밤색 있어요 \n']
p:  ['회색 한 장 있어요 \n']
_
q:  ['얼마 예요 \n']
a:  ['\t 구천 원 이에요 \n']
p:  ['17만 9천 원 이에요 \n']
_
q:  ['이 코트 는 얼마 예요 \n']
a:  ['\t 이 거 는 사만 원 에 드리겠습니다 \n']
p:  ['그 제품 도 삼만 원 이에요 \n']
_
q:  ['나이 가 어떻게 되시는데요 \n']
a:  ['\t 팔십 둘 이 요 \n']
p:  ['스물 여덟 인데 \n']
_
q:  ['그 몇 문 까지 신 을 수 있어요 \n']
a:  ['\t 이백육십 까지 신 을 수 있어요 \n']
p:  ['지금 110 까지 영업 나와요 \n']
_
q:  ['수면 양말 이건 얼마 예요 \n']
a:  ['\t 오천 원 이에요 \n']
p:  ['3만 원 짜 리도 있고 2만 원 짜 리도 있고 다양해요 \n']
_
q:  ['어른 들 는 얼마 정도 에요 \n']
a:  ['\t 만 오천 원 부터 비싼 거 는 사만 오천 원 까지 있어요 \n']
p:  ['이건 50 퍼센트 인데 할인 해서 40 고 이 게 7만 원 이에요 \n']
_
q